In [7]:
import pandas as pd
from datetime import datetime
import numpy as np

In [8]:
import pandas as pd
import re

In [9]:
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

In [10]:
pd.options.display.max_columns = None

In [11]:
df3 = pd.read_csv('data_processed_add_stems.csv', encoding = 'utf-16')

In [12]:
resumes2 = pd.read_csv('resumes_step2_stems.csv', encoding = 'utf-16')

In [7]:
resumes2.head(3)

,Unnamed: 0,birth_date,gender,area,education,title,education_level,salary.amount,salary.currency,specialization,skill_set,skills_desc,exp_length,exp_description,skills_desc_stem,exp_desc_stem
0,0,27 октября 1954,Мужчина,Москва,"[{'year': 1976, 'name': 'Высшее Военное Инжене...","Организатор корпоративных охот, рыбалок и эко-...",Высшее образование,60000.0,руб.,Руководитель проектов,"Работа в Фондах, НКО и ТПП, Союзах и Ассоциаци...",В работе: Аналитик.Трудоголик. Хобби: 50 лет ...,10915.0,Руководство административно-хозяйственной деят...,в работ аналитик трудоголик хобб лет занима ох...,руководств административн хозяйствен деятельн ...
1,1,7 июля 1978,Мужчина,Москва,"[{'year': 2002, 'name': 'Таганрогский Радиотех...",Руководитель проектов,Высшее образование,NaN,NaN,Руководитель проектов,Работоспособность,"Являюсь разносторонним человеком, люблю хорошо...",7554.0,"Организация рабочего процесса на объекте , вза...",явля разносторон человек любл хорош и плодотво...,организац рабоч процесс на объект взаимодейств...
2,2,20 января 1986,Мужчина,Москва,"[{'year': 2012, 'name': 'Институт Проблем Риск...",Руководитель проекта,Высшее образование,NaN,NaN,Руководитель проектов,"Автоматизация процессов,Проектное управление,О...","Системное мышление, глубокая аналитика и деком...",3732.0,Формирование бизнес требований. Сбор информац...,системн мышлен глубок аналитик и декомпозиц пр...,формирован бизнес требован сбор информац описа...


In [8]:
df3.head(3)

,Unnamed: 0,id,name,salary.from,salary.to,alternate_url,salary.currency,salary.gross,experience.name,experience.id,schedule.name,employment.name,employer.name,requirement,skills,description,prof_role,specializations,description_stem
0,0,67716839,Специалист по работе с покупателями,50000.0,NaN,https://hh.ru/vacancy/67716839,RUR,False,От 1 года до 3 лет,between1And3,Полный день,Полная занятость,ВкусВилл,ПК или ноутбук. Устойчивый интернет.,"Пользователь ПК,Грамотная речь,Поиск информаци...",Наша поддержка с душой реагирует на каждое обр...,Специалист технической поддержки,"Информационные технологии, интернет, телеком",наш поддержк с душ реагир на кажд обращен и вс...
1,1,68539555,Frontend разработчик (junior/middle),60000.0,120000.0,https://hh.ru/vacancy/68539555,RUR,False,От 1 года до 3 лет,between1And3,Полный день,Полная занятость,Инфоурок,"HTML(5)/CSS(3), Flex, кроссбраузерность. Препр...","JavaScript,Git,jQuery,HTML5,Sass",О нас: Продуктовая компания (основной продукт...,"Программист, разработчик","Информационные технологии, интернет, телеком",о нас продуктов компан основн продукт сайт inf...
2,2,68731055,Корректор контента,80000.0,NaN,https://hh.ru/vacancy/68731055,RUR,True,От 1 года до 3 лет,between1And3,Удаленная работа,Полная занятость,Российская академия народного хозяйства и госу...,"Знание Microsoft Office (PowerPoint, Word, Exc...","Грамотность,Грамотная речь,Работа в команде,Де...",В команду по развитию онлайн-образования требу...,"Копирайтер, редактор, корректор","Искусство, развлечения, масс-медиа;Информацион...",в команд по развит онлайн образован треб специ...


In [13]:
df3['skills'] = df3['skills'].str.replace(',', ' ').str.lower().fillna('--')

# Обучение модели

In [11]:
sentences = list(df3['skills'])
new_sentences = [list(i.split(' ')) for i in sentences] 
sentences = new_sentences

In [12]:
%time

model = FastText(vector_size=300, window=3, min_count=1)  
model.build_vocab(sentences)

model.train(corpus_iterable=sentences, total_examples=model.corpus_count, epochs=10)  # train

Wall time: 0 ns


(2360375, 3293970)

In [15]:
# Размер - 2 GB

model.save("fasttext_skills.model")

In [ ]:
model = Word2Vec.load("fasttext_skills.model")

# Применение модели

In [16]:

def match_skills(df3, skills):
    
    df3['skills'] = df3['skills'].str.replace(',', ' ').str.lower().fillna('--')    
    
    w2v_model = Word2Vec.load("fasttext_skills.model")
    
    sentences = list(df3['skills'])
    new_sentences = [list(i.split(' ')) for i in sentences] 
    sentences = new_sentences

    target_sentence = skills.replace(',', ' ').lower()
    target_sentence

    sentences_similarity = np.zeros(len(sentences))
    indexes = np.zeros(len(sentences))

    target_sentence_words = [w for w in target_sentence.split(' ') if w in w2v_model.wv.index_to_key]

    for idx, sentence in enumerate(sentences):
        sentence_words = [w for w in sentence if w in w2v_model.wv.index_to_key]

        sim = w2v_model.wv.n_similarity(target_sentence_words, sentence_words)
        sentences_similarity[idx] = sim

        ss = ' '.join(sentence)
        indexes[idx] = df3[df3['skills']==ss].index[0]

    result = list(zip(indexes, sentences_similarity, sentences))
    result.sort(key=lambda item:item[1], reverse=True)
    print("Target:", target_sentence)
    
    df = pd.DataFrame(zip(indexes, sentences_similarity), columns=['Indexes', 'Skill_Similarity'])
    
    
    ids = []

    for x in result[:5]:
        ids.append (x[0])
    
    return (ids)

# Пример поиска вакансий по скиллам

In [17]:
# выберем скиллы из датасета резюме

target_sentence = resumes2.loc[5, 'skill_set'].replace(',', ' ').lower()
target_sentence

'диагностика текущих процессов разработка стратегии развития руководство маркетинговыми исследования системный анализ управление персоналом анализ бизнес показателей организаторские навыки пользователь пк ведение переговоров работа в команде деловое общение управление проектами руководство коллективом'

In [18]:
# на всех вакансиях

print (datetime.now())

ids = match_skills(df3, 'диагностика текущих процессов разработка стратегии развития руководство \
маркетинговыми исследования системный анализ управление персоналом анализ бизнес показателей организаторские \
навыки пользователь пк ведение переговоров работа в команде деловое общение управление проектами руководство коллективом')

print (datetime.now())

2022-08-29 22:58:49.445673
Target: диагностика текущих процессов разработка стратегии развития руководство маркетинговыми исследования системный анализ управление персоналом анализ бизнес показателей организаторские навыки пользователь пк ведение переговоров работа в команде деловое общение управление проектами руководство коллективом
2022-08-29 23:03:05.069407


In [19]:
df3[df3.index.isin(ids) == True][['name', 'alternate_url', 'skills']].style.hide_index()

name,alternate_url,skills
Исполнительный Вице-президент Глобал (Executive VP Global),https://hh.ru/vacancy/68106874,управление командой оптимизация бизнес-процессов бюджетирование бизнес-планирование проведение презентаций стратегическое планирование операционное управление навыки переговоров руководство коллективом английский язык умение работать в условиях многозадачности
Руководитель управления закупок ГК,https://hh.ru/vacancy/68466276,управление проектами работа в команде ведение переговоров организаторские навыки управление закупками централизация закупок трансформация процессов английский язык стратегическое планирование стратегическое мышление
Руководитель проектов развития продуктового маркетинга (CRM),https://hh.ru/vacancy/68620402,деловая коммуникация развитие продаж деловое общение организаторские навыки linux управление проектами crm анализ бизнес показателей системное мышление умение работать в условиях многозадачности
Проект менеджер / Product manager,https://hh.ru/vacancy/68792490,управление персоналом ведение переговоров организаторские навыки работа в команде управление проектами управление командой управление продажами развитие продаж анализ бизнес показателей мотивация персонала деловое общение стратегическое мышление анализ финансовых показателей тайм-менеджмент
Креативный директор,https://hh.ru/vacancy/68644053,управление персоналом управление проектами fashion чувство стиля съемочный процесс генерация идей fashion trends лидерство конкурентный анализ креативность организаторские навыки обучение персонала работа в команде руководство коллективом проведение презентаций деловая коммуникация


In [20]:
# на всех вакансиях

print (datetime.now())

ids = match_skills(df3, 'python Postgres docker')

print (datetime.now())

2022-08-29 23:09:20.669779
Target: python postgres docker
2022-08-29 23:15:46.262562


In [21]:
df3[df3.index.isin(ids) == True][['name', 'alternate_url', 'skills']].style.hide_index()

name,alternate_url,skills
Python разработчик (Junior),https://hh.ru/vacancy/68101643,python http docker-compose sqlalchemy
Стажер в лабораторию искусственного интеллекта Сбера,https://hh.ru/vacancy/67887990,python pandas pytorch machine learning
Анатилик/Data scientist (графы),https://hh.ru/vacancy/68525095,python spark machine learning pytorch tensorflow
Backend разработчик,https://hh.ru/vacancy/68849481,python django framework postgresql celery docker
Python Developer,https://hh.ru/vacancy/68365436,linux python docker ci/cd docker-compose
